# Cell Transmission Model (CTM) Simulator

This notebook demonstrates the CTM traffic simulation implementation with interactive examples.

## Overview

The Cell Transmission Model (CTM) is a discrete approximation of the Lighthill-Whitham-Richards (LWR) traffic flow model. It discretizes the freeway into cells and tracks the flow of vehicles between cells over time.

### Key Features:
- **Triangular Fundamental Diagram**: Relates flow, density, and speed
- **On-ramp Merging**: Models vehicles entering from on-ramps
- **Time-varying Demand**: Supports dynamic traffic patterns
- **Lane Drops/Additions**: Handles capacity changes
- **Ramp Metering**: Controls on-ramp flow rates

In [ ]:
# Import required modules
from ctm_simulation import (
    CellConfig,
    CTMSimulation,
    OnRampConfig,
    build_uniform_mainline,
    run_basic_scenario,
)

# Try to import plotting libraries
try:
    import matplotlib.pyplot as plt
    import numpy as np
    HAS_PLOTTING = True
except ImportError:
    print("Note: Install matplotlib and numpy for visualizations")
    HAS_PLOTTING = False

## Example 1: Basic Freeway Simulation

We start with a simple 5-cell freeway with constant parameters and time-varying demand.

In [ ]:
# Create a 5-cell mainline
cells = build_uniform_mainline(
    num_cells=5,
    cell_length_km=0.5,
    lanes=3,
    free_flow_speed_kmh=100.0,
    congestion_wave_speed_kmh=20.0,
    capacity_veh_per_hour_per_lane=2200.0,
    jam_density_veh_per_km_per_lane=160.0,
)

print(f"Created {len(cells)} cells, each {cells[0].length_km} km long")
print(f"Total freeway length: {sum(c.length_km for c in cells)} km")
print(f"Capacity per lane: {cells[0].capacity_veh_per_hour_per_lane} veh/h/lane")
print(f"Total capacity: {cells[0].capacity_veh_per_hour_per_lane * cells[0].lanes} veh/h")

In [ ]:
# Define time-varying demand (triangular profile)
def triangular_demand(step: int) -> float:
    """Ramp up, peak, then ramp down demand pattern."""
    peak = 6000.0  # Near capacity
    if step < 10:
        return peak * (step / 10.0)
    elif step < 20:
        return peak
    else:
        return max(0.0, peak - 300.0 * (step - 20))

# Create and run simulation
sim = CTMSimulation(
    cells=cells,
    time_step_hours=0.05,  # 3 minutes
    upstream_demand_profile=triangular_demand,
    downstream_supply_profile=7000.0,  # High downstream capacity
)

result = sim.run(steps=40)

print(f"\nSimulation completed:")
print(f"  Duration: {result.duration_hours:.2f} hours")
print(f"  Time steps: {result.steps - 1}")
print(f"  Time step size: {result.time_step_hours * 60:.1f} minutes")

In [ ]:
# Plot results
if HAS_PLOTTING:
    fig, axes = plt.subplots(2, 1, figsize=(12, 8))
    
    time = result.time_vector()
    
    # Plot densities
    ax = axes[0]
    for cell_name in sorted(result.densities.keys()):
        ax.plot(time, result.densities[cell_name], label=cell_name, linewidth=2)
    ax.set_xlabel('Time (hours)')
    ax.set_ylabel('Density (veh/km/lane)')
    ax.set_title('Cell Densities Over Time')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    
    # Plot flows
    ax = axes[1]
    intervals = result.interval_vector()
    interval_times = [t[0] for t in intervals]
    
    for cell_name in sorted(result.flows.keys()):
        ax.plot(interval_times, result.flows[cell_name], label=cell_name, linewidth=2)
    ax.set_xlabel('Time (hours)')
    ax.set_ylabel('Flow (veh/h)')
    ax.set_title('Cell Outflows Over Time')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("Install matplotlib to see plots")
    print(f"\nFinal densities: {[f'{d:.1f}' for d in [result.densities[f'cell_{i}'][-1] for i in range(5)]]}")

## Example 2: Freeway with On-Ramp

Now we add an on-ramp to see how merging traffic affects mainline operations.

In [ ]:
# Create mainline with lane drop
cells = build_uniform_mainline(
    num_cells=4,
    cell_length_km=0.5,
    lanes=[3, 3, 2, 2],  # Lane drop at cell 2
    free_flow_speed_kmh=100.0,
    congestion_wave_speed_kmh=20.0,
    capacity_veh_per_hour_per_lane=2200.0,
    jam_density_veh_per_km_per_lane=160.0,
)

# On-ramp with time-varying demand
def ramp_demand(step: int) -> float:
    """Rush hour pattern on the ramp."""
    if step < 10:
        return 200.0
    elif step < 25:
        return 800.0  # Heavy ramp demand
    else:
        return 300.0

on_ramp = OnRampConfig(
    target_cell=2,
    arrival_rate_profile=ramp_demand,
    mainline_priority=0.6,  # Favor mainline
    initial_queue_veh=5.0,
)

print(f"Created {len(cells)} cells with lane configuration: {[c.lanes for c in cells]}")
print(f"On-ramp merges at cell 2")
print(f"Mainline priority: {on_ramp.mainline_priority}")

In [ ]:
# Run simulation
sim = CTMSimulation(
    cells=cells,
    time_step_hours=0.05,
    upstream_demand_profile=4500.0,  # Moderate mainline demand
    downstream_supply_profile=5000.0,
    on_ramps=[on_ramp],
)

result = sim.run(steps=35)

# Print ramp statistics
ramp_name = list(result.ramp_queues.keys())[0]
max_queue = max(result.ramp_queues[ramp_name])
avg_queue = sum(result.ramp_queues[ramp_name]) / len(result.ramp_queues[ramp_name])
total_ramp_flow = sum(result.ramp_flows[ramp_name]) * result.time_step_hours

print(f"\nOn-ramp statistics:")
print(f"  Max queue: {max_queue:.1f} vehicles")
print(f"  Avg queue: {avg_queue:.1f} vehicles")
print(f"  Total vehicles merged: {total_ramp_flow:.0f} vehicles")

In [ ]:
# Plot results
if HAS_PLOTTING:
    fig, axes = plt.subplots(3, 1, figsize=(12, 10))
    
    time = result.time_vector()
    
    # Plot densities
    ax = axes[0]
    for idx, cell_name in enumerate(sorted(result.densities.keys())):
        lane_count = cells[idx].lanes
        label = f"{cell_name} ({lane_count}L)"
        ax.plot(time, result.densities[cell_name], label=label, linewidth=2)
    ax.set_xlabel('Time (hours)')
    ax.set_ylabel('Density (veh/km/lane)')
    ax.set_title('Cell Densities (with Lane Configuration)')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    
    # Plot ramp queue and flow
    ax = axes[1]
    ax.plot(time, result.ramp_queues[ramp_name], 'r-', linewidth=2, label='Queue')
    ax.set_xlabel('Time (hours)')
    ax.set_ylabel('Queue Length (vehicles)', color='r')
    ax.tick_params(axis='y', labelcolor='r')
    ax.grid(True, alpha=0.3)
    
    ax2 = ax.twinx()
    intervals = result.interval_vector()
    interval_times = [t[0] for t in intervals]
    ax2.plot(interval_times, result.ramp_flows[ramp_name], 'b-', linewidth=2, label='Flow')
    ax2.set_ylabel('Ramp Flow (veh/h)', color='b')
    ax2.tick_params(axis='y', labelcolor='b')
    ax.set_title('On-Ramp Queue and Flow')
    
    # Plot mainline flows
    ax = axes[2]
    for cell_name in sorted(result.flows.keys()):
        ax.plot(interval_times, result.flows[cell_name], label=cell_name, linewidth=2)
    ax.set_xlabel('Time (hours)')
    ax.set_ylabel('Flow (veh/h)')
    ax.set_title('Mainline Outflows')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("Install matplotlib to see plots")

## Example 3: Ramp Metering

Compare scenarios with and without ramp metering to see its effect on queue buildup and mainline operations.

In [ ]:
# Setup common parameters
cells = build_uniform_mainline(
    num_cells=3,
    cell_length_km=0.5,
    lanes=3,
    free_flow_speed_kmh=100.0,
    congestion_wave_speed_kmh=20.0,
    capacity_veh_per_hour_per_lane=2200.0,
    jam_density_veh_per_km_per_lane=160.0,
)

# Scenario A: No metering
ramp_no_meter = OnRampConfig(
    target_cell=1,
    arrival_rate_profile=1200.0,
    meter_rate_veh_per_hour=None,  # No metering
    mainline_priority=0.5,
)

sim_no_meter = CTMSimulation(
    cells=cells,
    time_step_hours=0.1,
    upstream_demand_profile=5000.0,
    on_ramps=[ramp_no_meter],
)

result_no_meter = sim_no_meter.run(steps=20)

# Scenario B: With metering
ramp_with_meter = OnRampConfig(
    target_cell=1,
    arrival_rate_profile=1200.0,
    meter_rate_veh_per_hour=600.0,  # Metering rate
    mainline_priority=0.5,
)

sim_with_meter = CTMSimulation(
    cells=cells,
    time_step_hours=0.1,
    upstream_demand_profile=5000.0,
    on_ramps=[ramp_with_meter],
)

result_with_meter = sim_with_meter.run(steps=20)

# Compare results
ramp_name = list(result_no_meter.ramp_queues.keys())[0]

print(f"\nComparison:")
print(f"  No metering - Final queue: {result_no_meter.ramp_queues[ramp_name][-1]:.1f} veh")
print(f"  With metering - Final queue: {result_with_meter.ramp_queues[ramp_name][-1]:.1f} veh")

total_flow_no_meter = sum(result_no_meter.ramp_flows[ramp_name])
total_flow_with_meter = sum(result_with_meter.ramp_flows[ramp_name])

print(f"  No metering - Total ramp flow: {total_flow_no_meter:.0f} veh")
print(f"  With metering - Total ramp flow: {total_flow_with_meter:.0f} veh")

In [ ]:
# Plot comparison
if HAS_PLOTTING:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    time_no = result_no_meter.time_vector()
    time_with = result_with_meter.time_vector()
    
    ramp_name_no = list(result_no_meter.ramp_queues.keys())[0]
    ramp_name_with = list(result_with_meter.ramp_queues.keys())[0]
    
    # Queue comparison
    ax = axes[0]
    ax.plot(time_no, result_no_meter.ramp_queues[ramp_name_no], 'r-', linewidth=2, label='No metering')
    ax.plot(time_with, result_with_meter.ramp_queues[ramp_name_with], 'b-', linewidth=2, label='With metering')
    ax.set_xlabel('Time (hours)')
    ax.set_ylabel('Queue Length (vehicles)')
    ax.set_title('On-Ramp Queue Comparison')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    
    # Flow comparison
    ax = axes[1]
    intervals_no = result_no_meter.interval_vector()
    intervals_with = result_with_meter.interval_vector()
    interval_times_no = [t[0] for t in intervals_no]
    interval_times_with = [t[0] for t in intervals_with]
    
    ax.plot(interval_times_no, result_no_meter.ramp_flows[ramp_name_no], 'r-', linewidth=2, label='No metering')
    ax.plot(interval_times_with, result_with_meter.ramp_flows[ramp_name_with], 'b-', linewidth=2, label='With metering')
    ax.set_xlabel('Time (hours)')
    ax.set_ylabel('Ramp Flow (veh/h)')
    ax.set_title('On-Ramp Flow Comparison')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("Install matplotlib to see plots")

## Interactive Experiment

Try creating your own scenario! Modify the parameters below and run the cells to see the results.

In [ ]:
# CUSTOMIZE THESE PARAMETERS
NUM_CELLS = 5
CELL_LENGTH_KM = 0.5
NUM_LANES = 3
FREE_FLOW_SPEED = 100.0  # km/h
UPSTREAM_DEMAND = 5000.0  # veh/h
SIMULATION_HOURS = 2.0
TIME_STEP_HOURS = 0.05  # 3 minutes

# On-ramp parameters (set to None to disable)
ONRAMP_TARGET_CELL = 2
ONRAMP_DEMAND = 600.0  # veh/h
ONRAMP_METER_RATE = None  # veh/h (None = no metering)
MAINLINE_PRIORITY = 0.7  # 0 to 1 (higher = favor mainline)

# Create and run simulation
cells = build_uniform_mainline(
    num_cells=NUM_CELLS,
    cell_length_km=CELL_LENGTH_KM,
    lanes=NUM_LANES,
    free_flow_speed_kmh=FREE_FLOW_SPEED,
    congestion_wave_speed_kmh=20.0,
    capacity_veh_per_hour_per_lane=2200.0,
    jam_density_veh_per_km_per_lane=160.0,
)

on_ramps = []
if ONRAMP_TARGET_CELL is not None:
    on_ramps.append(OnRampConfig(
        target_cell=ONRAMP_TARGET_CELL,
        arrival_rate_profile=ONRAMP_DEMAND,
        meter_rate_veh_per_hour=ONRAMP_METER_RATE,
        mainline_priority=MAINLINE_PRIORITY,
    ))

sim = CTMSimulation(
    cells=cells,
    time_step_hours=TIME_STEP_HOURS,
    upstream_demand_profile=UPSTREAM_DEMAND,
    on_ramps=on_ramps if on_ramps else None,
)

steps = int(SIMULATION_HOURS / TIME_STEP_HOURS)
result = sim.run(steps=steps)

print(f"Simulation complete: {result.duration_hours:.2f} hours, {result.steps - 1} time steps")

In [ ]:
# Plot your custom scenario
if HAS_PLOTTING:
    n_plots = 3 if on_ramps else 2
    fig, axes = plt.subplots(n_plots, 1, figsize=(12, 4*n_plots))
    if n_plots == 2:
        axes = [axes[0], axes[1], None]
    
    time = result.time_vector()
    intervals = result.interval_vector()
    interval_times = [t[0] for t in intervals]
    
    # Densities
    ax = axes[0]
    for cell_name in sorted(result.densities.keys()):
        ax.plot(time, result.densities[cell_name], label=cell_name, linewidth=2)
    ax.set_xlabel('Time (hours)')
    ax.set_ylabel('Density (veh/km/lane)')
    ax.set_title('Cell Densities')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    
    # Flows
    ax = axes[1]
    for cell_name in sorted(result.flows.keys()):
        ax.plot(interval_times, result.flows[cell_name], label=cell_name, linewidth=2)
    ax.set_xlabel('Time (hours)')
    ax.set_ylabel('Flow (veh/h)')
    ax.set_title('Cell Outflows')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    
    # Ramp queue (if present)
    if on_ramps and axes[2] is not None:
        ax = axes[2]
        for ramp_name in result.ramp_queues.keys():
            ax.plot(time, result.ramp_queues[ramp_name], 'r-', linewidth=2, label='Queue')
        ax.set_xlabel('Time (hours)')
        ax.set_ylabel('Queue Length (vehicles)')
        ax.set_title('On-Ramp Queue')
        ax.legend(loc='best')
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("Install matplotlib to see plots")
    print(f"\nFinal densities: {[result.densities[f'cell_{i}'][-1] for i in range(NUM_CELLS)]}")

## Summary

This notebook demonstrated:
1. Basic freeway simulation with time-varying demand
2. On-ramp merging with queue dynamics
3. Ramp metering strategies
4. Interactive experimentation

For more examples, see `examples_ctm.py` in the Exercise2 directory.

### Further Reading
- Daganzo, C. F. (1994). "The cell transmission model: A dynamic representation of highway traffic consistent with the hydrodynamic theory." Transportation Research Part B, 28(4), 269-287.
- Munoz, L., et al. (2003). "A traffic state estimation method for freeways." Transportation Research Part C, 11(3-4), 247-270.